In [12]:
import os
import numpy as np
import tensorflow as tf
keras = tf.keras

In [22]:
import sys
sys.setrecursionlimit(1000000)  # Set to a higher value based on your needs

import tensorflow_datasets as tfds
import matplotlib.pyplot as plt  # Import matplotlib for displaying images

tfds.disable_progress_bar()

try:
    # Split the data manually into 80% training, 10% testing, 10% validation
    (raw_train, raw_validation, raw_test), metadata = tfds.load(
        'cats_vs_dogs',
        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
        with_info=True,
        as_supervised=True,
    )
    print(metadata)

    # Create a function object that we can use to get labels
    get_label_name = metadata.features['label'].int2str

    # Display 2 images from the dataset
    for image, label in raw_train.take(2):
        plt.figure()
        plt.imshow(image)
        plt.title(get_label_name(label))
        plt.show()
        
    #Data Preprocessing
    IMG_SIZE = 160 # All images will be resized to 160*160

    def format_example(image, label):
        """
        returns an image that is reshaped to IMG_SIZE
        """
        image = tf.cast(image, tf.float32)
        image = (image/127.5) - 1
        image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
        return image, label
    
    #Applying above function to all the images using map
    train = raw_train.map(format_example)
    validation = raw_validation.map(format_example)
    test = raw_test.map(format_example)
    
    for image, label in train.take(2):
        plt.figure()
        plt.imshow(image)
        plt.title(get_label_name(label))
    
    #Shuffling & Batching Images
    BATCH_SIZE = 32
    SHUFFLE_BUFFER_SIZE = 1000
    
    train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    validation_batches = validation.batch(BATCH_SIZE)
    test_batches = test.batch(BATCH_SIZE)
    
    #Shape of an Original Image vs New Image
    for img, label in raw_train.take(2):
        print("Original Shape: ", img.shape)
    
    for img, label in train.take(2):
        print("New Shape: ", img.shape)
        
    #Picking a Pretrained Model
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
    #Create the base model from the pre-trained model MobileNet V2
    base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,
                                                   include_top = False,
                                                   weights = 'imagenet')
    
    #This model is trained on 1.4 million images and has 1000 different classes.
    
    print(base_model.summary())
    
    # At this point, this base_model will simply output a shape (32,5,5,1280) tensor that is a feature extraction frmo our original (1,160,160,3) image.
    # The 32 means that we have 32 different layers of filters/features. 
    
    for image, _ in train_batches.take(1):
        pass
    feature_batch = base_model(image)
    print(feature_batch.shape)
    
    
    base_model.trainable = False
    print(base_model.summary())
    
    
    # Adding our Classifier
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

    # Adding the prediction layer that will be a single dense neuron. 
    prediction_layer = keras.layers.Dense(1)

    
except RecursionError as e:
    print("RecursionError:", e)


RecursionError: maximum recursion depth exceeded while calling a Python object
